In [34]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
from math import pow
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from bitstring import BitArray

In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [3]:
secret_mes = bytes(input("Enter the message you want to hide. Only use ASCII characters"),"ascii")

## Encryption

In [4]:
key_len = 16
#keygen
key = get_random_bytes(key_len)
cipher = AES.new(key, AES.MODE_GCM)
#encrypt
ciphertext, tag = cipher.encrypt_and_digest(secret_mes)
encode_this = BitArray(cipher.nonce + ciphertext + tag)

In [5]:
encode_this

BitArray('0xe565a7a911662998c8c2e4d333ced161b7a1fd698b5e6f2d19e8e57bcf5ea576b09f435d11c1dee60187dfe8')

## Encode

In [67]:
result = "Dear audience,"
current_tokens = tokenizer(result, return_tensors="pt")
#print(current_tokens.input_ids)
next_token_logits = model(**current_tokens).logits[:, -1, :]
filtered_next_token_logits = torch.softmax(top_k_top_p_filtering(next_token_logits, top_p=0.999)[0],0)
print(filtered_next_token_logits.shape)
l_index = 0
r_index = len(filtered_next_token_logits)
j=0
while len(filtered_next_token_logits) > 1:
    b = encode_this[j]

    acc = 0
    i = 0
    if not b:
        while acc<0.5:
            acc += filtered_next_token_logits[i]
            i += 1
        
        r_index -= len(filtered_next_token_logits)-i
        filtered_next_token_logits = torch.softmax(filtered_next_token_logits[:i],0)
    else:
        while acc<0.5:
            acc += filtered_next_token_logits[-i]
            i += 1

        l_index += len(filtered_next_token_logits)-i
        filtered_next_token_logits = torch.softmax(filtered_next_token_logits[-i:],0)
    print(float(acc),i,filtered_next_token_logits.shape,j)
    print("lr index", l_index, r_index)
    j+=1


torch.Size([50257])
0.5000057220458984 49494 torch.Size([49494]) 0
lr index 763 50257
0.5000150203704834 24746 torch.Size([24746]) 1
lr index 25511 50257
0.5000158548355103 12373 torch.Size([12373]) 2
lr index 37884 50257
0.5000550746917725 6187 torch.Size([6187]) 3
lr index 37884 44071
0.5000677108764648 3094 torch.Size([3094]) 4
lr index 37884 40978
0.5 1547 torch.Size([1547]) 5
lr index 39431 40978
0.5003232359886169 774 torch.Size([774]) 6
lr index 39431 40205
0.5000001788139343 387 torch.Size([387]) 7
lr index 39818 40205
0.5012921690940857 194 torch.Size([194]) 8
lr index 39818 40012
0.5 97 torch.Size([97]) 9
lr index 39915 40012
0.5051546096801758 49 torch.Size([49]) 10
lr index 39963 40012
0.510204017162323 25 torch.Size([25]) 11
lr index 39963 39988
0.5199999213218689 13 torch.Size([13]) 12
lr index 39963 39976
0.5384615659713745 7 torch.Size([7]) 13
lr index 39969 39976
0.5714285969734192 4 torch.Size([4]) 14
lr index 39969 39973
0.5 2 torch.Size([2]) 15
lr index 39971 39973


In [65]:
len(filtered_next_token_logits)

1

In [ ]:
current_string = "Dear audience,"
previous_string = ""

i = 0
while i < len(encode_this):
    b = encode_this[i]
    print("Current string:", current_string)
    current_tokens = tokenizer(current_string, return_tensors="pt")
    print(current_tokens.input_ids)
    probs = torch.topk(nn.functional.softmax(model(**current_tokens).logits[:, -1, :], dim=-1),100)
    (indeces_0, indeces_1) = balance_probs(probs)

    if b:
        (indeces_right, indeces_wrong) = (indeces_1, indeces_0)
    else:
        (indeces_right, indeces_wrong) = (indeces_0, indeces_1)
    
    print("Don't use these tokens ", [tokenizer.decode(probs.indices[0,i]) for i in indeces_wrong])
    print("Prefer these tokens ", [tokenizer.decode(probs.indices[0,i]) for i in indeces_right])
    
    new_token_string = input("add another token")
    new_token = tokenizer(new_token_string).input_ids[0]

    print([probs.indices[0,i] for i in indeces_right], [probs.indices[0,i] for i in indeces_wrong])

    if new_token in [probs.indices[0,i] for i in indeces_right]:
        i += 1
        print(i)
    elif new_token in [probs.indices[0,i] for i in indeces_wrong]:
        print("token from the 'don't use set'")
        continue

    previous_string = current_string[:]
    current_string += new_token_string

## Decode

In [ ]:
in_string = "Dear audience, please do your own due and take"
seed_len = 3
in_tokens = tokenizer(in_string, return_tensors="pt").input_ids

In [ ]:
bs = []

for i in range(seed_len, in_tokens.size()[1]):
    current_tokens = in_tokens[:,:i]
    probs = torch.topk(nn.functional.softmax(model(current_tokens).logits[:,-1, :], dim=-1),8)
    indeces = balance_probs(probs)
    new_token = in_tokens[0,i]

    if new_token in [probs.indices[0,i] for i in indeces[0]]:
        bs.append(False)
    elif new_token in [probs.indices[0,i] for i in indeces[1]]:
        bs.append(True)

decrypt_this = BitArray(bs).tobytes()

## Decryption

In [ ]:
nonce = decrypt_this[:key_len]
ciphertext = decrypt_this[key_len:-key_len]
tag = decrypt_this[-key_len:]

In [ ]:
#decrypt
cipher = AES.new(key, AES.MODE_GCM, nonce)
cipher.decrypt_and_verify(ciphertext, tag)